In [1]:
def UserInputs2WorkingForm(n,N,Bd,Pd,phi,SampleFrequency,FileOfInterest,HomeDirectory,directory,TrainingDataFile):
    x = {
        'n': n,
        'N': N,
        'Bd': Bd,
        'Pd': Pd,
        'Phi': phi,
        'Sampling Frequency': SampleFrequency,
        'File of Interest': FileOfInterest,
        'HomeDirectory': HomeDirectory,
        'Working Directory': directory,
        'TrainingFileName': TrainingDataFile
    }
    return x

In [2]:
def GetTrainingData(UserInputs):
    for file in UserInputs['Working Directory']:
        if file == UserInputs['TrainingFileName']:
            dataset = pd.read_csv(file,header = 0,index_col = 0)

    X = dataset.values[:,1:(dataset.shape[1]-1)]
    Y = dataset.values[:,0]
    validation_size = 0.20
    seed = 6
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed) 
    
    return X_train, X_test, Y_train, Y_test

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#BEGIN HARDCODING OF GUI INFO
n = 2000 / 60
N = 16
Bd = 0.331*254
Pd = 2.815*254
phi = 15.17 * np.pi / 180
SampleFrequency = 20000
FileOfInterest = '2003.10.22.12.06.24'
HomeDirectory = os.getcwd()
os.chdir(HomeDirectory)
directory = os.listdir(HomeDirectory)
TrainingDataFile = "DELETE.csv"
#END OF HARDCODING OF GUI INFO




In [4]:
def TrainModel(X_train,Y_train):
    classifier = RandomForestClassifier(n_estimators=1000)
    classifier = classifier.fit(X_train, Y_train)
    return classifier

In [5]:
def PredictModel(classifier,X_test):
    Y_test_pred = classifier.predict(X_test)
    return Y_test_pred

In [6]:
def GetData(UserInput):
    data = pd.read_table(UserInput['File of Interest'],header = None)
    data.columns = ['b1x','b1y','b2x','b2y','b3x','b3y','b4x','b4y']
    return np.transpose(data.values[:,0])

In [7]:
from Functions import getCompleteDataFrame
from Functions import getGraphs
from Functions import getTESTDataFrame

UserInput = UserInputs2WorkingForm(n,N,Bd,Pd,phi,SampleFrequency,FileOfInterest,HomeDirectory,directory,TrainingDataFile)
X_train, X_test, Y_train, Y_test = GetTrainingData(UserInput)
classifier = TrainModel(X_train, Y_train)
Y_test_pred = PredictModel(classifier,X_test)
b1x = GetData(UserInput)

NumberOfSamples = len(b1x)
dt = 1/SampleFrequency
Tmax = dt*NumberOfSamples
TEST = getTESTDataFrame(b1x,Tmax,NumberOfSamples,SampleFrequency,n,N,Bd,Pd,phi)
TEST1 = TEST.values[:,0:(TEST.shape[1]-1)]
OUTCOME = PredictModel(classifier,TEST1)
file = 3
figs = getGraphs(b1x,Tmax,NumberOfSamples,SampleFrequency,n,N,Bd,Pd,phi,file,1)

[0.0000000e+00 1.0240000e+00 2.0480000e+00 ... 2.0968448e+04 2.0969472e+04
 2.0970496e+04]
[ 1.34774633e+02  6.66845395e+01 -1.52599389e+00 ... -1.15808611e-02
 -2.02875287e-03  3.81786751e-03]


In [8]:
print(b1x)
print(OUTCOME)
print("Accuracy on training set is : {}".format(classifier.score(X_train, Y_train)))
print("Accuracy on test set is : {}".format(classifier.score(X_test, Y_test)))
print(classification_report(Y_test, Y_test_pred))

[-0.022 -0.105 -0.183 ... -0.193 -0.115 -0.042]
['Early']
Accuracy on training set is : 1.0
Accuracy on test set is : 0.9281159420289855
                         precision    recall  f1-score   support

                  Early       0.98      0.92      0.95       276
       Imminent Failure       1.00      0.75      0.86        20
     Inner Race Failure       0.90      1.00      0.95         9
                 Normal       0.91      0.97      0.94       879
Rolling Element Failure       0.98      0.94      0.96        63
        Stage 2 Failure       0.94      0.98      0.96        64
                Suspect       0.93      0.85      0.89       414

            avg / total       0.93      0.93      0.93      1725

